In [1]:
from scripts.software_frs import frs_traceability2
FRS = frs_traceability2
from officelib.xllib import *
from officelib.const import xlconst as xlc
import sys
import os
import re

In [2]:
_frs_match = re.compile(r"(.*FRS\d+)\.?([\d\.]*)").match
_toplevel_match = re.compile(r"^\>?[\+\*]{2}(FRS\d+)[\+\*]{2}(.*)$").match
_subitem_match = re.compile(r"^\>?[\+\*]+\s\*(FRS[\d\.]+)\:?\*\:?(.*)$").match
_canceled_match = re.compile(r"^[\+\*]+\s\-\*(FRS[\d\.]+)\:?\*\:?(.*)\-").match
_header_match = re.compile(r"^\>?[\+\*]{2}([\d\.\w]+)[\+\*]{2}").match

In [3]:
_item_match = re.compile(r"^\>?[\*]*\s*[\+\*]+(FRS[\d\.]+)\:?[\+\*]*\:?(.*)$").match

In [4]:
def _extract_frs_lines(lines):
    start_matches = (
        (_item_match, 0),
        (_canceled_match, FRS.FRS_NA),
    )
    end_matches = (
        _item_match,
        _canceled_match,
    )
    i = 0
    while True:
        try:
            line = lines[i]
        except IndexError:
            break
        i += 1
        for match, flags in start_matches:
            done = False
            if not line:
                m = None
            else:
                m = match(line)
            if m:
                f = m.group(1)
                t = m.group(2).strip()
                l = [t]
                while True:
                    try:
                        line = lines[i]
                    except IndexError:
                        done = True
                        break
                    if any(m(line) for m in end_matches):
                        done = True
                        break
                    l.append(" " + line.strip())
                    i += 1
                text = "\n".join(l)
                # if l and not t:
                #     text = "\n" + text
                yield f, flags, text
            if done:
                break

In [5]:
def t(s):
    if not _item_match(s):
        print("FAIL:", s)
t("* *FRS1234*:")
t("*+FRS1234.1+*")

In [6]:
_cache = None

In [76]:
# caching scheme for test purposes. 

def _lffi():
    global _cache
    if _cache is None:
        issues = FRS._download_issues()
        _cache = issues
    else:
        issues = _cache
    relevant = FRS.filter_relevant_issues(issues)
    all_frs = {(None, 0, "")}
    for v in relevant:
        lines = v.description.splitlines()
        for frs in _extract_frs_lines(lines):
            all_frs.add(frs)
    all_frs.remove((None, 0, ""))
    return all_frs
    
def load_frs_from_issuetracker():
    return _lffi()

def filter_relevant_issues(issues):
    relevant = []
    for v in issues.values():
        if v.sprint_milestone == "Legacy" and v.status not in ("Closed", "Rejected"):
                relevant.append(v)
    return relevant

frs_traceability2.load_frs_from_issuetracker = load_frs_from_issuetracker
FRS.filter_relevant_issues = filter_relevant_issues

In [77]:
def ws_data(ws):
    cr = ws.Cells.Range
    first = cr("A2")
    last_col = cr("A1").End(xlc.xlToRight).Column
    last = first.End(xlc.xlDown).Offset(1, last_col)
    return cr(first, last).Value

def split_frs(frs):
    return [f for f in frs.split() if f]

def load_test_list(user_tests):
    xl = Excel(visible=False)
    with HiddenXl(xl):
        wb = xl.Workbooks.Open(user_tests)
        ws = wb.Worksheets(1)
        id_test, name, purpose, start, steps, accept, frs = \
                        frs_traceability2.find_cols(ws, 
                            "ID_TEST", "NAME", "PURPOSE", "STARTING POINT", 
                            "STEPS", "ACCEPTANCE_CRITERIA", "List Web FRS")
        data = ws_data(ws)
    
    wb.Close(False)
    xl.Quit()
    
    res = []
    for row in data:
        i = row[id_test]
        n = row[name]
        p = row[purpose]
        sta = row[start]
        ste = row[steps]
        a = row[accept]
        f = row[frs]
        if not f: continue
        res.append((i, n, p, sta, ste, a, split_frs(f)))
    res.sort(key=lambda t: t[0])
    return res

In [78]:
def node_data(node):
    return node.id, node.text.strip()

def make_frs_line(items):
    lines = []
    items2 = sorted(items, key=frs_traceability2.sort_frs_item)
    for id, line in items2:
        if not line:
            lines.append("%s" % id)
        else:
            lines.append("%s: %s" % (id, line.rstrip()))
    return "\n".join(lines)

def ifmt(i):
    if int(i) == i:
        return "%d" % i
    return "%s" % i

def write_test(reqs, test, file):
    tmplt = _frs_tmplt
    i, name, p, sta, ste, a, frs = test
    items = set()
    for f in frs:
        node = reqs.lookup(f)
        if not node: raise ValueError(f)
        if node.is_na(): continue
        items.add((node.id, node.text))
        indent = node.id.count(".")
        for n in node.iter():
            nindent = n.id.count(".") - indent
            items.add((" " * nindent + n.id, n.text))
    frs_line = make_frs_line(items)
    s = tmplt % (ifmt(i), name, frs_line, p.rstrip(), sta.rstrip(), ste.rstrip(), a.rstrip())
    print(s, file=file)

In [79]:
try:
    _reqs
except NameError:
    _reqs = None

In [80]:
def make_paste_data(root):
    data = [("FRS Number", "Level", "id_test", "Tested", "Is Leaf?", "Is Holdup?")]
    di = data[0].index("Tested") + 1
    for i, node in enumerate(root.iter(), 2):
        f = node.id
        tests = FRS.TEST_ITEM_SEP.join(str(i) for i in node.get_tests())
        if node.is_leaf():
            if node.is_tested():
                tested = "Y"
                holdup = ""
            elif node.is_na():
                tested = "n/a"
                holdup = ""
            else:
                tested = ""
                holdup = "Y"
            leaf = "Y"
        else:
            if node.is_na():
                tested = "n/a"
            else:
                tested = FRS._xl_child_yes(i, di, node.total_len())
            leaf = ""
            holdup = ""
            
        if not tests and node.has_children() and node.is_tested():
            tests = "'--"
        
        if f[:3] == "3.0":
            count = 0
        else:
            count = f.count(".")
        data.append((f, count, tests, tested, leaf, holdup))
    return data

In [81]:
from officelib.xllib import *

def main(user_tests):
    xl = Excel()
    all_frs_items = FRS.load_frs_from_issuetracker()
    reqs = FRS.build_frs_tree(all_frs_items)

    with screen_lock(xl):
        print("Loading User Test Matrix...")
        FRS.process_user_tests(xl, reqs, user_tests)

    print("Compiling data for final matrix")
    data = make_paste_data(reqs)
    
    ws = FRS.get_matrix_sheet(xl)
    FRS.paste_data(ws, data)
    print("Done")
    

In [82]:
trace_path = "C:\\Users\\Nathan\\Documents\\PBS\\SW test\\FRS Project"
user_tests = "FRS Tests 180827 1.xlsx"
p1 = os.path.join(trace_path, user_tests)
main(p1)

Loading User Test Matrix...
Compiling data for final matrix
Pasting test data
Applying alignment formatting
Marking untested cells
Applying column autofit
Applying row autofit
Done


In [31]:
all_frs_items = FRS.load_frs_from_issuetracker()
reqs = FRS.build_frs_tree(all_frs_items)
reqs

Root('')